In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.stats import reciprocal
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt

import seaborn as sns
sns.set(font_scale=2.2)
plt.style.use('seaborn')

import gc
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold, train_test_split, ShuffleSplit
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
import shap
import time

import os
print(os.listdir("../input"))

### Helper functions

In [ ]:
def one_hot_encoder(df):
    ohe = OneHotEncoder(categorical_features='all', sparse=False, categories="auto")
    df_encoded = ohe.fit_transform(df)
    return pd.DataFrame(df_encoded)

In [ ]:
def area_region(df):
    df['a1r1'] = df['area1']*df['lugar1']
    df['a1r2'] = df['area1']*df['lugar2']
    df['a1r3'] = df['area1']*df['lugar3']
    df['a1r4'] = df['area1']*df['lugar4']
    df['a1r5'] = df['area1']*df['lugar5']
    df['a1r6'] = df['area1']*df['lugar6']
    
    df['a2r1'] = df['area2']*df['lugar1']
    df['a2r2'] = df['area2']*df['lugar2']
    df['a2r3'] = df['area2']*df['lugar3']
    df['a2r4'] = df['area2']*df['lugar4']
    df['a2r5'] = df['area2']*df['lugar5']
    df['a2r6'] = df['area2']*df['lugar6']
    return df

In [ ]:
def hh_civil(df):
    df['hh_civil_1'] = df['parentesco1'] * df['estadocivil2']
    df['hh_civil_2'] = df['parentesco1'] * df['estadocivil3']
    df['hh_civil_3'] = df['parentesco1'] * df['estadocivil4']
    df['hh_civil_4'] = df['parentesco1'] * df['estadocivil5']
    df['hh_civil_5'] = df['parentesco1'] * df['estadocivil6']
    df['hh_civil_6'] = df['parentesco1'] * df['estadocivil7']
    # Civil status numbers by household
    df['civil_1'] = df['estadocivil4'] + df['estadocivil5'] + df['estadocivil6']
    temp_df = df['civil_1'].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'civil_1': 'divorced_plus'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")  

    temp_df = df['estadocivil1'].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'estadocivil1': 'child_10'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")

    temp_df = df['estadocivil2'].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'estadocivil2': 'coupled'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")

    temp_df = df['estadocivil3'].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'estadocivil3': 'married'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")

    temp_df = df['estadocivil7'].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'estadocivil7': 'single'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")

    column_list=['divorced_plus', 'single', 'coupled', 'child_10', 'married']
    return df, column_list

In [ ]:
def assets_by_hh_alt(df):
    df['qmob_hh'] = df['qmobilephone'] / df['tamhog']
    df['v18q1_hh'] = df['v18q1'] / df['tamhog']
    df['computer_hh'] = df['computer_y'] / df['tamhog']
    df['television_hh'] = df['television_y'] / df['tamhog']
    df['rooms_hh'] = df['rooms'] / df['tamhog']
    df['bedrooms_hh'] = df['bedrooms'] / df['tamhog']
    l1=list(df)
    # Rooms by different demographic groups
    df = df.assign(child_rooms=np.where(df['r4t1'] != 0, df['rooms'] / df['r4t1'], 99))
    df = df.assign(mchild_rooms=np.where(df['r4h1'] != 0, df['rooms'] / df['r4h1'], 99))
    df = df.assign(fchild_rooms=np.where(df['r4m1'] != 0, df['rooms'] / df['r4m1'], 99))
    df = df.assign(fadult_rooms=np.where(df['adul_female_hh'] != 0, df['rooms'] / df['adul_female_hh'], 99))
    df = df.assign(madult_rooms=np.where(df['adul_male_hh'] != 0, df['rooms'] / df['adul_male_hh'], 99))
    df = df.assign(mayor_rooms=np.where(df['hogar_mayor'] != 0, df['rooms'] / df['hogar_mayor'], 99))
     # Computers by different demographic groups
    df = df.assign(child_comp=np.where(df['r4t1'] != 0, df['computer_y'] / df['r4t1'], 99))
    df = df.assign(mchild_comp=np.where(df['r4h1'] != 0, df['computer_y'] / df['r4h1'], 99))
    df = df.assign(fchild_comp=np.where(df['r4m1'] != 0, df['computer_y'] / df['r4m1'], 99))
    df = df.assign(fadult_comp=np.where(df['adul_female_hh'] != 0, df['computer_y'] / df['adul_female_hh'], 99))
    df = df.assign(madult_comp=np.where(df['adul_male_hh'] != 0, df['computer_y'] / df['adul_male_hh'], 99))
    df = df.assign(mayor_comp=np.where(df['hogar_mayor'] != 0, df['computer_y'] / df['hogar_mayor'], 99))

    l2 = list(df)
    column_list = [col for col in l2 if col not in l1]
    return df, column_list

In [ ]:
def demographics_2(df):
    df['out_hh'] = df['tamhog'] - df['tamviv']
    df['adult'] = df['hogar_adul'] - df['hogar_mayor']
    # Number of adult males in household
    temp_df = df['male'][df['age']>19].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'male': 'adul_male_hh'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")
    df['adul_male_hh']=df['adul_male_hh'].fillna(0)
    # Number of adult females in household
    temp_df = df['female'][df['age']>19].groupby(df['idhogar']).sum().to_frame()
    temp_df.rename(columns={'female': 'adul_female_hh'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")
    df['adul_female_hh']=df['adul_female_hh'].fillna(0)
    # Household heads and gender
    df["male_hh"] = df["parentesco1"] * df["male"]
    df["female_hh"] = df["parentesco1"] * df["female"]
    
    l1 = list(df)
    df = df.assign(child_adult=np.where(df['adult'] != 0, df['hogar_nin'] / df['adult'], 99)) 
    df = df.assign(old_adult=np.where(df['adult'] != 0, df['hogar_mayor'] / df['adult'], 99)) 
    df = df.assign(r4h1_adult=np.where(df['adult'] != 0, df['r4h1'] / df['adult'], 99))
    df = df.assign(r4m1_adult=np.where(df['adult'] != 0, df['r4m1'] / df['adult'], 99))
    df = df.assign(r4t1_adult=np.where(df['adult'] != 0, df['r4t1'] / df['adult'], 99))
    # Number of disabled women in household
    df['dis_female'] = df['dis'] * df['female']
    temp_df = df['dis_female'].groupby(df['idhogar']).sum().to_frame()
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")
    df['dis_female_y']=df['dis_female_y'].fillna(0)
    df = df.assign(dis_female_adult_fem=np.where(df['adul_female_hh'] != 0, df['dis_female_y'] / df['adul_female_hh'], 99))
    df = df.assign(dis_female_adult_mal=np.where(df['adul_male_hh'] != 0, df['dis_female_y'] / df['adul_male_hh'], 99))
    l2 = list(df)
    column_list = [col for col in l2 if col not in l1]
    
    return df, column_list    

In [ ]:
def remove_squared(df):
    df = df[[x for x in list(df) if not x.startswith('SQB')]]
    df = df.drop(['agesq'], axis=1)
    return df

In [ ]:
def remove_corr_features(df):
    corr_matrix = df.corr()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.975)]
    df = df.drop(columns = to_drop)
    return df

In [ ]:
def clean_dataset(df):
    temp_df = df['escolari'][df['age']>19].groupby(df['idhogar']).mean().to_frame()
    temp_df.rename(columns={'escolari': 'meaneduc_new'}, inplace=True)
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")
    df['meaneduc_new']= df['meaneduc_new'].fillna(df['meaneduc_new'].mean())
    df.loc[df['v18q1'].isnull(), 'v18q1'] = 0
    df['depend_alt'] = np.sqrt(df['SQBdependency'])
    df['educ_male_hh'] = np.sqrt(df['SQBedjefe']).astype(int)
    df['educ_female_hh'] = df.edjefa
    df.loc[df['educ_female_hh']=="yes", 'educ_female_hh'] = 1
    df.loc[df['educ_female_hh']=="no", 'educ_female_hh'] = 0
    df['educ_female_hh'] = df['educ_female_hh'].astype(int)
    df.loc[(df["parentesco1"]==1) & (df["female"]==1), "educ_male_hh"] = 99
    df.loc[df["parentesco1"]==0, "educ_male_hh"] = 99
    df.loc[(df["parentesco1"]==1) & (df["male"]==1), "educ_female_hh"] = 99
    df.loc[df["parentesco1"]==0, "educ_female_hh"] = 99
    # Creating categorical variables for household head female education
    df["fhh_educ_cat1"] = 0
    df.loc[df.educ_female_hh < 6, "fhh_educ_cat1"] = 1
    df["fhh_educ_cat2"] = 0
    df.loc[(df.educ_female_hh >= 6) & (df.educ_female_hh < 12), "fhh_educ_cat2"] = 1
    df["fhh_educ_cat3"] = 0
    df.loc[(df.educ_female_hh >= 12) & (df.educ_female_hh < 99), "fhh_educ_cat3"] = 1
    # Creating categorical variables for household head male education
    df["mhh_educ_cat1"] = 0
    df.loc[df.educ_male_hh < 6, "mhh_educ_cat1"] = 1
    df["mhh_educ_cat2"] = 0
    df.loc[(df.educ_male_hh >= 6) & (df.educ_male_hh < 12), "mhh_educ_cat2"] = 1
    df["mhh_educ_cat3"] = 0
    df.loc[(df.educ_male_hh >= 12) & (df.educ_male_hh < 99), "mhh_educ_cat3"] = 1
    df["v2a1"] = df["v2a1"].fillna(0)
    df["rez_esc"] = df["rez_esc"].fillna(-1)
    temp_df = df["computer"].groupby(df["idhogar"]).sum().to_frame()
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")
    temp_df = df["television"].groupby(df["idhogar"]).sum().to_frame()
    df = df.merge(temp_df, left_on="idhogar", right_index=True, how="outer")
    return df

### Preparing the dataset

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
df = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
df.shape

In [ ]:
df = clean_dataset(df)
df = area_region(df)
df, column_list_1 = demographics_2(df)
df, column_list_2 = hh_civil(df)
df, column_list_3 = assets_by_hh_alt(df)
df = remove_squared(df)

In [ ]:
column_list = column_list_1 + column_list_2 + column_list_3
one_hot_df = one_hot_encoder(df[column_list])
df = pd.concat([df, one_hot_df], axis=1)
df = df.drop(column_list, axis=1)
df.shape

In [ ]:
df = df.drop(['idhogar', 'meaneduc', 'dependency', 'edjefe', 'edjefa',
              'educ_male_hh', 'educ_female_hh'], axis=1)
missing = pd.DataFrame(df.isnull().sum())
missing['Proportion'] = missing/len(df)
missing.columns=["Missing", "Proportion"]
missing[missing["Missing"] > 0]

In [ ]:
df = remove_corr_features(df)
df["Target"] = df["Target"] - 1
df.shape

In [ ]:
train_set = df[df['Target'].notnull()]
train_set = train_set.ix[:,~train_set.columns.duplicated()]
test_set = df[df['Target'].isnull()]
test_set = test_set.ix[:,~test_set.columns.duplicated()]
train_set.shape, test_set.shape

In [ ]:
X_train=train_set.drop(['Id','Target'], axis=1)
y_train=train_set['Target']
X_test=test_set.drop(['Id','Target'], axis=1)
y_test=test_set['Target']

In [ ]:
from imblearn.over_sampling import ADASYN
balanced = ADASYN(random_state = 199)
X_train, y_train = balanced.fit_sample(X_train, y_train)
X_train=pd.DataFrame(X_train)
y_train=pd.DataFrame(y_train)
X_train.shape, y_train.shape

In [ ]:
temp=pd.DataFrame(y_train)
temp[0].value_counts()/len(temp)

In [ ]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
                    ('imputer', Imputer(strategy = 'median')),
                    ('scaler', MinMaxScaler())]
                   )
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.fit_transform(X_test)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:
def print_execution_time(start):
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print('*'*20, "Execution ended in {:0>2}h {:0>2}m {:05.2f}s".format(int(hours),int(minutes),seconds), '*'*20)

### Random Forest

In [ ]:
def RFC_OOF(params, N_FOLDs, SEED=1989):
    final_rfc = RandomForestClassifier(n_estimators = params['n_estimators'],
                                       max_features = params['max_features'],
                                       max_depth = params['max_depth'],
                                       min_samples_split = params['min_samples_split'],
                                       min_samples_leaf = params['min_samples_leaf'],
                                       bootstrap = params['bootstrap'],
                                       n_jobs=-1,
                                       verbose=3,
                                       random_state = 1989
                                      )

    kfold = N_FOLDs
    kf = StratifiedKFold(n_splits=kfold, shuffle=True)
    predicts_result = []
    predicts_train = []
    for i, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
        print('='*30, '{} of {} folds'.format(i+1, kfold), '='*30)
        start = time.time()
        train, val = X_train.iloc[train_index], X_train.iloc[test_index]
        train_y, val_y = y_train.iloc[train_index], y_train.iloc[test_index]
        final_rfc.fit(train, train_y)
        predicts_result.append(final_rfc.predict(X_test))
        predicts_train.append(final_rfc.predict(X_train))
        print_execution_time(start)
    return predicts_result, predicts_train

In [ ]:
final_params = {
          'n_estimators': 2000,
          'max_features': 'sqrt',
          'max_depth': 110,
          'min_samples_split': 2,
          'min_samples_leaf': 1,
          'bootstrap': True,
        }

In [ ]:
N_Folds = 10
SEED = 1989
predicts_result, predicts_train = RFC_OOF(final_params, N_Folds, SEED=1989)

In [ ]:
predicts_result_array = np.transpose(np.array(predicts_result))
df_pred = pd.DataFrame(predicts_result_array)
df_pred.columns = ['rfc_1', 'rfc_2', 'rfc_3', 'rfc_4', 'rfc_5', 'rfc_6', 'rfc_7', 'rfc_8', 'rfc_9', 'rfc_10']
for col in df_pred.columns:
    df_pred[col] = df_pred[col].round().astype(int) + 1

In [ ]:
predicts_train_array = np.transpose(np.array(predicts_train))
df_pred_train = pd.DataFrame(predicts_train_array)
df_pred_train.columns = ['rfc_1', 'rfc_2', 'rfc_3', 'rfc_4', 'rfc_5', 'rfc_6', 'rfc_7', 'rfc_8', 'rfc_9', 'rfc_10']
for col in df_pred_train.columns:
    df_pred_train[col] = df_pred_train[col].round().astype(int) + 1

### XGBoost

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
final_params = {'max_depth': 15, 
                'min_child_weight': 3, 
                'subsample': 0.8, 
                'colsample_bytree': 0.7, 
                'gamma': 0.0156,
                'silent': 1, 
                'seed': 1989, 
                'objective': 'multi:softmax', 
                'learning_rate': 0.1, 
                'n_estimators': 1000}
final_params['num_class'] = 4
num_boost_round = 2000

In [ ]:
def XGB_OOF(params, N_FOLDs, SEED=1989):
    final_xgb = XGBClassifier(objective = params['objective'],
                              n_estimators = params['n_estimators'],
                              colsample_bytree = params['colsample_bytree'],
                              subsample = params['subsample'],
                              max_depth = params['max_depth'],
                              min_child_weight = params['min_child_weight'],
                              gamma = params['gamma'],
                              learning_rate = params['learning_rate'],
                              num_class = params['num_class'],
                              silent = params['silent'],
                              n_jobs=-1,
                              verbose=3,
                              random_state = 42,
                              seed = params['seed']
                              )

    kfold = N_FOLDs
    kf = StratifiedKFold(n_splits=kfold, shuffle=True)
    predicts_result = []
    predicts_train = []
    for i, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
        print('='*30, '{} of {} folds'.format(i+1, kfold), '='*30)
        start = time.time()
        train, val = X_train.iloc[train_index], X_train.iloc[test_index]
        train_y, val_y = y_train.iloc[train_index], y_train.iloc[test_index]
        final_xgb.fit(train, train_y, eval_set=[(train, train_y), (val, val_y)], eval_metric='mlogloss',
                    early_stopping_rounds=100, verbose=1)
        predicts_result.append(final_xgb.predict(X_test))
        predicts_train.append(final_xgb.predict(X_train))
        print_execution_time(start)
    return predicts_result, predicts_train

In [ ]:
N_Folds = 10
SEED = 1989
predicts_result, predicts_train = XGB_OOF(final_params, N_Folds, SEED=1989)

In [ ]:
predicts_result_array = np.transpose(np.array(predicts_result))
xgb_results = pd.DataFrame(predicts_result_array)
xgb_results.columns = ['xgb_1', 'xgb_2', 'xgb_3', 'xgb_4', 'xgb_5', 'xgb_6', 'xgb_7', 'xgb_8', 'xgb_9', 'xgb_10']
for col in xgb_results.columns:
    xgb_results[col] = xgb_results[col].round().astype(int) + 1
df_pred = pd.concat([df_pred, xgb_results], axis=1)

In [ ]:
predicts_train_array = np.transpose(np.array(predicts_train))
xgb_pred_train = pd.DataFrame(predicts_train_array)
xgb_pred_train.columns = ['xgb_1', 'xgb_2', 'xgb_3', 'xgb_4', 'xgb_5', 'xgb_6', 'xgb_7', 'xgb_8', 'xgb_9', 'xgb_10']
for col in xgb_pred_train.columns:
    xgb_pred_train[col] = xgb_pred_train[col].round().astype(int) + 1
df_pred_train = pd.concat([df_pred_train, xgb_pred_train], axis=1)

### LightGBM

In [ ]:
def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [ ]:
def LGB_OOF(params, N_FOLDs, SEED=1989):
    clf = lgb.LGBMClassifier(objective='multiclass',
                             random_state=42,
                             max_depth=params['max_depth'], 
                             learning_rate=params['learning_rate'],  
                             silent=True, 
                             metric='multi_logloss',
                             n_jobs=-1, n_estimators=15000, 
                             class_weight='balanced',
                             colsample_bytree = params['colsample_bytree'], 
                             min_split_gain= params['min_split_gain'], 
                             bagging_freq = params['bagging_freq'],
                             min_child_weight=params['min_child_weight'],
                             num_leaves = params['num_leaves'], 
                             subsample = params['subsample'],
                             reg_alpha= params['reg_alpha'],
                             reg_lambda= params['reg_lambda'],
                             num_class=len(np.unique(y_train.values.ravel())),
                             bagging_seed=params['bagging_seed'],
                             seed=SEED,
                            )

    kfold = N_FOLDs
    kf = StratifiedKFold(n_splits=kfold, shuffle=True)
    predicts_result = []
    predicts_train = []

    for i, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
        print('='*30, '{} of {} folds'.format(i+1, kfold), '='*30)
        start = time.time()
        train, val = X_train.iloc[train_index], X_train.iloc[test_index]
        train_y, val_y = y_train.iloc[train_index], y_train.iloc[test_index]
        clf.fit(train, train_y, eval_set=[(train, train_y), (val, val_y)], eval_metric=evaluate_macroF1_lgb,
                early_stopping_rounds=500, verbose=5)
        predicts_result.append(clf.predict(X_test))
        predicts_train.append(clf.predict(X_train))
        print_execution_time(start)
    return predicts_result, predicts_train

In [ ]:
params = {'application': 'multiclass',
          'metric': 'multi_logloss',
          'num_class': 4,
          'class_weight': 'balanced',
          'num_leaves': 36,
          'max_depth': 5,
          'min_child_weight': 8,
          'min_split_gain': 0.002962792245398255,
          'colsample_bytree': 0.3541243866073689,
          'subsample': 0.9154409603974555,
          'bagging_freq': 1,
          'bagging_seed': 2,
          'reg_alpha': 0.90874782008929,
          'reg_lambda': 0.1828455057465017,
          'learning_rate': 0.01931493659729515,
          'seed': 1989,
          'verbose': 1}

In [ ]:
N_Folds = 10
SEED = 1989
predicts_result, predicts_train = LGB_OOF(params, N_Folds, SEED=1989)

In [ ]:
predicts_result_array = np.transpose(np.array(predicts_result))
lgb_results = pd.DataFrame(predicts_result_array)
lgb_results.columns = ['lgb_1', 'lgb_2', 'lgb_3', 'lgb_4', 'lgb_5', 'lgb_6', 'lgb_7', 'lgb_8', 'lgb_9', 'lgb_10']
for col in lgb_results.columns:
    lgb_results[col] = lgb_results[col].round().astype(int) + 1
df_pred = pd.concat([df_pred, lgb_results], axis=1)

In [ ]:
predicts_train_array = np.transpose(np.array(predicts_train))
lgb_pred_train = pd.DataFrame(predicts_train_array)
lgb_pred_train.columns = ['lgb_1', 'lgb_2', 'lgb_3', 'lgb_4', 'lgb_5', 'lgb_6', 'lgb_7', 'lgb_8', 'lgb_9', 'lgb_10']
for col in lgb_pred_train.columns:
    lgb_pred_train[col] = lgb_pred_train[col].round().astype(int) + 1
df_pred_train = pd.concat([df_pred_train, lgb_pred_train], axis=1)

### Neural net

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam

In [ ]:
dims = df_pred_train.shape[1]
# Defining the model
model = Sequential()
model.add(Dense(64, input_dim=dims))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

In [ ]:
df_pred_train_copy = df_pred_train.copy()
df_pred_train_copy = np.array(df_pred_train_copy)
df_pred_train_copy.shape

In [ ]:
y_train_copy = y_train.copy()
y_train_copy = np.array(y_train_copy).astype(int)
y_train_copy = keras.utils.to_categorical(y_train_copy, num_classes=4)
y_train_copy.shape

In [ ]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(df_pred_train_copy, 
          y_train_copy,
          epochs=50,
          batch_size=32)

In [ ]:
predictions = model.predict(np.array(df_pred))
pred_list = [np.argmax(pred) + 1 for pred in predictions]

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test_set['Id'].reset_index(drop=True)
submission['Target'] = pred_list
submission.to_csv('submission_lasttry.csv', index = False)

df_pred['Target'] = df_pred.mean(axis=1).round().astype(int)
df_pred = pd.DataFrame(df_pred['Target'])